#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 mydata and use them to predict the outcome of cases from the last 30 days

In [139]:
! pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable


In [140]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=18804 sha256=a95b4ade78d02258485078b998758cf61cbb26aa2fa9e105942121a1ead46662
  Stored in directory: /tmp/pip-ephem-wheel-cache-jpxtt_g_/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [141]:
import os
import pandas as pd
import numpy as np
import pickle
import re
import sys
import time

In [142]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree, Boston311KerasNLP


## Load extra features

In [143]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

#format tomorrows date as yyyy-mm-dd
tomorrows_date =  now + pd.DateOffset(days=1)
tomorrows_datestring = tomorrows_date.strftime("%Y-%m-%d")

In [144]:
print('today_datestring: ', today_datestring)
print('tomorrows_datestring: ', tomorrows_datestring)

today_datestring:  2023-10-14
tomorrows_datestring:  2023-10-15


In [145]:
#set path to mydata
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_service_id.csv'
#json_file = './daily_models/Boston311KerasNLP/20230925_143704_Boston311KerasNLP.json'
#model_file = './daily_models/Boston311KerasNLP/20230925_143704_Boston311KerasNLP.h5'
#json_file = './daily_models/Boston311KerasNLP/20231003_031227_Boston311KerasNLP.json'
#model_file = './daily_models/Boston311KerasNLP/20231003_031227_Boston311KerasNLP.h5'
json_file = './daily_models/Boston311KerasNLP/20231011_223042_Boston311KerasNLP.json'
model_file = './daily_models/Boston311KerasNLP/20231011_223042_Boston311KerasNLP.h5'
json_file = './daily_models/Boston311KerasNLP/20231011_232344_Boston311KerasNLP.json'
model_file = './daily_models/Boston311KerasNLP/20231011_232344_Boston311KerasNLP.h5'

json_file = './daily_models/Boston311KerasNLP/20231012_005720_Boston311KerasNLP.json'
model_file = './daily_models/Boston311KerasNLP/20231012_005720_Boston311KerasNLP.h5'

kerasNLP_model = Boston311KerasNLP()
kerasNLP_model.load( json_file, model_file)
kerasNLP_model.predict_date_range['end'] = tomorrows_datestring


In [146]:
#load data from all_311_data.csv
mydata = pd.read_csv('all_311_cases.csv', low_memory=False)

In [147]:
data = kerasNLP_model.load_data( data=mydata, train_or_predict='predict' )
data = kerasNLP_model.enhance_data( data, 'predict')

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['closed_dt'] = pd.to_datetime(data['closed_dt'])
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['open_dt'] = pd.to_datetime(data['open_dt'])
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [148]:
#show first ten records with newest open_dt
data.sort_values(by=['open_dt'], ascending=False).head(10)

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,...,location_street_name,location_zipcode,latitude,longitude,geom_4326,source,survival_time,event,ward_number,survival_time_hours
2721369,101005106692,2023-10-12 20:03:50,NaN,NaT,ONTIME,Open,,Loud Parties/Music/People,Boston Police Department,Noise Disturbance,...,174 Saint Alphonsus St,2120.0,42.331330,-71.102201,0101000020E6100000CAEF68748AC651C011C7F801692A...,Constituent Call,NaT,0,10,NaN
2721368,101005106690,2023-10-12 20:03:20,NaN,NaT,ONTIME,Open,,Loud Parties/Music/People,Boston Police Department,Noise Disturbance,...,51 Gardner St,2134.0,42.353540,-71.128471,0101000020E61000000DC2D0DC38C851C0C75938C9402D...,Constituent Call,NaT,0,21,NaN
2721367,101005106688,2023-10-12 20:01:00,NaN,NaT,ONTIME,Open,,Loud Parties/Music/People,Boston Police Department,Noise Disturbance,...,140 Sutherland Rd,2135.0,42.340850,-71.147531,0101000020E6100000A94E8C2471C951C095F502F6A02B...,Constituent Call,NaT,0,21,NaN
2721366,101005106686,2023-10-12 19:58:07,2023-10-16 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,41-43 Saratoga St,2128.0,42.376069,-71.037460,0101000020E61000006FC008BE65C251C09CF9F3032330...,Citizens Connect App,NaT,0,1,NaN
2721365,101005106685,2023-10-12 19:53:12,2023-10-16 04:30:00,NaT,ONTIME,Open,,Requests for Street Cleaning,Public Works Department,Street Cleaning,...,3 Bakersfield St,2125.0,42.317985,-71.060800,0101000020E610000005279327E4C351C0DF0495B9B328...,Citizens Connect App,NaT,0,13,NaN
2721364,101005106684,2023-10-12 19:51:18,2023-10-17 04:30:00,NaT,ONTIME,Open,,Improper Storage of Trash (Barrels),Public Works Department,Code Enforcement,...,15 Appleton St,2116.0,42.346230,-71.071231,0101000020E6100000D958620A8FC451C0AFAFDE3F512C...,Citizens Connect App,NaT,0,5,NaN
2721363,101005106683,2023-10-12 19:48:47,2023-10-16 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,INTERSECTION Ponce Way & Mcgreevey Way,NaN,42.334865,-71.097623,0101000020E6100000D411B7723FC651C0F67747DCDC2A...,Citizens Connect App,NaT,0,10,NaN
2721362,101005106681,2023-10-12 19:25:56,2023-10-17 04:30:00,NaT,ONTIME,Open,,Improper Storage of Trash (Barrels),Public Works Department,Code Enforcement,...,5-7 Harriet St,2135.0,42.353160,-71.154621,0101000020E610000023CD1C4EE5C951C0920FD555342D...,Citizens Connect App,NaT,0,22,NaN
2721361,101005106680,2023-10-12 19:16:44,2023-10-27 04:30:00,NaT,ONTIME,Open,,Street Light Outages,Public Works Department,Street Lights,...,87 Waverly St,2135.0,42.361870,-71.140651,0101000020E61000006EC3796B00C951C074E171BE512E...,Citizens Connect App,NaT,0,22,NaN
2721360,101005106679,2023-10-12 19:08:00,2023-10-16 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,45 Harbor View St,2125.0,42.316970,-71.054561,0101000020E6100000989473EB7DC351C06A0D09759228...,Constituent Call,NaT,0,13,NaN


In [149]:
clean_data = kerasNLP_model.clean_data_for_prediction( data )

In [150]:
clean_data.head()

,case_enquiry_id,queue_ANML02_LostFound,queue_ANML_General,queue_BHA_General,queue_BPD_Administrative,queue_BPD_Districts,queue_BPS_Administrative,queue_BPS_Transportation Administration,queue_BTDT_AVRS Interface Queue,queue_BTDT_Abandoned Bicycle,...,department_ECON,department_GEN_,department_GRNi,department_INFO,department_ISD,department_No Q,department_ONS_,department_PARK,department_PROP,department_PWDx
0,101005047896,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,101005047898,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,101005047900,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,101005047901,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,101005047903,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [151]:
mydata = clean_data

In [152]:
mydata['case_enquiry_id']

0        101005047896
1        101005047898
2        101005047900
3        101005047901
4        101005047903
             ...     
27785    101005106685
27786    101005106686
27787    101005106688
27788    101005106690
27789    101005106692
Name: case_enquiry_id, Length: 27790, dtype: object

In [153]:
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle

pickle_file = 'dataframe.pkl'

X = None

if os.path.exists(pickle_file):
    #check if the file date is earlier than EXTRA_mydata_FILE date
    pickle_file_date = os.path.getmtime(pickle_file)
    EXTRA_mydata_FILE_date = os.path.getmtime(EXTRA_mydata_FILE)
    if pickle_file_date < EXTRA_mydata_FILE_date:
        os.remove(pickle_file)

if os.path.exists(pickle_file):

    X = pickle.load(open(pickle_file, "rb"))
else:
    X = pd.read_csv(EXTRA_mydata_FILE)

    #rename service_request_id to case_enquiry_id
    X.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X = X[X['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    X['cls_embedding'] = X['cls_embedding'].apply(literal_eval).apply(np.array)
    X['pooled_embedding'] = X['pooled_embedding'].apply(literal_eval).apply(np.array)

    pickle.dump(X, open(pickle_file, "wb"))



In [154]:
#concatenate the two dataframes and reindex
df = X

In [155]:
df.shape

(265388, 3)

In [156]:

# Assuming df is your DataFrame and it has columns 'cls_embedding' and 'pooled_embedding'
cls_embedding_flattened = np.stack(df['cls_embedding'].to_numpy())
pooled_embedding_flattened = np.stack(df['pooled_embedding'].to_numpy())

# Remove the old columns
df.drop(['cls_embedding', 'pooled_embedding'], axis=1, inplace=True)

# Add the new flattened columns
df_cls = pd.DataFrame(cls_embedding_flattened, columns=[f'cls_{i}' for i in range(cls_embedding_flattened.shape[1])])
df_pooled = pd.DataFrame(pooled_embedding_flattened, columns=[f'pooled_{i}' for i in range(pooled_embedding_flattened.shape[1])])

df = pd.concat([df, df_cls, df_pooled], axis=1)

In [157]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
is_numeric = df['case_enquiry_id'].str.isnumeric()

In [158]:
df = df[is_numeric]

In [159]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')

In [160]:
df.shape

(265388, 257)

In [161]:
mydata.shape

(27790, 244)

In [162]:
#join them so we are left only with records that have mydata in both files
new_mydata = mydata.merge(df, on='case_enquiry_id', how='inner')


In [163]:
new_mydata.shape

(18364, 500)

In [164]:

df = new_mydata

In [165]:
#cast all columns that are type bool to float
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype('float64')
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('float64') 

In [166]:
#list the number of rows in X and y
print(df.dtypes)


case_enquiry_id              object
queue_ANML02_LostFound      float64
queue_ANML_General          float64
queue_BHA_General           float64
queue_BPD_Administrative    float64
                             ...   
pooled_123                  float64
pooled_124                  float64
pooled_125                  float64
pooled_126                  float64
pooled_127                  float64
Length: 500, dtype: object


In [167]:
#free all unused dataframes
#df_to_delete = [cls_embedding_flattened, pooled_embedding_flattened, df_cls, df_pooled, X, new_mydata, is_numeric, mydata]

#for data_frame in df_to_delete:
#    if data_frame is not None:
#        del data_frame

In [168]:
case_enquiry_id = df['case_enquiry_id']
X_predict = df.drop(['case_enquiry_id'], axis=1)


In [169]:

#parse CLS embedding column as array
predictions = kerasNLP_model.model.predict(X_predict)

574/574 [==============================] - 1s 1ms/step


In [170]:
# Define a function to flatten an array into a string.
def array_to_string(arr):
    return ' '.join(map(str, arr))

# Apply the function along axis 1 (rows).
string_predictions = np.apply_along_axis(array_to_string, axis=1, arr=predictions)

# Now string_predictions is a 1D NumPy array where each element is a string
# that contains all the elements from the corresponding row in the original 2D array.
print(string_predictions)

['0.69621736 0.1048159 0.09348048 0.050427526 0.018703781 0.009535758 0.0030683808 0.0009248479 0.0018980113 0.0022288996 0.0013717008 0.001206683 0.0017248009 0.0008430812 0.00015550887 0.00015258412 0.00015963984 0.00019606862 0.0009300194 0.0009923542 0.0005723305 7.313432e-05 8.34998e-05 5.0312392e-05 9.674421e-05 0.00036280087 0.0014697359 0.0013923218 0.0011302679 6.340503e-05 0.0002478269 0.0012345643 2.7883067e-05 0.00029076386 4.3574895e-05 0.00053975236 0.0011071641 0.0008230719 5.6481924e-05 2.0655905e-05 4.8104437e-05 0.0006139021 2.9946788e-05 2.1597238e-05 3.0264906e-05 8.709836e-06 1.4010528e-05 2.8864682e-05 1.2374675e-05 1.514752e-05 5.3457015e-06 1.9114743e-05 5.1529023e-06 5.5953383e-06 9.467195e-06 1.8367135e-05 1.4351918e-05 1.5381422e-05 1.2469389e-05 4.2625747e-06 9.794217e-06 4.9479845e-06 1.3144313e-05 1.620517e-05 1.4512815e-05 1.568347e-06 4.8774605e-06 1.2552658e-05 6.931033e-06 1.9850217e-05 3.1672716e-06 3.0747385e-06 1.6639302e-06 3.3245124e-06 1.6594709e

In [171]:
#combine case_enquiry_id and predictions into a dataframe
predictions_df = pd.DataFrame({'case_enquiry_id':case_enquiry_id, 'prediction':string_predictions})

In [172]:
bin_labels = [
    "0-12 hours",      # Less than half a day
    "12-24 hours",     # Half to one day
    "1-3 days",        # One to three days
    "4-7 days",        # Four to seven days
    "1-2 weeks",       # One to two weeks
    "2-4 weeks",       # Two to four weeks
    "1-2 months",      # One to two months
    "2-4 months",      # Two to four months
    "4+ months"        # More than four months
]

#predictions_df['prediction'] = predictions_df['prediction'].apply(lambda x: bin_labels[x])

In [173]:
len(predictions_df)

18364

In [174]:
print(predictions)

[[6.96217358e-01 1.04815900e-01 9.34804827e-02 ... 1.13095178e-08
  4.00462937e-07 9.58609326e-06]
 [5.04953749e-02 1.28426617e-02 8.68148450e-03 ... 2.96081929e-03
  3.06316232e-03 3.91401649e-01]
 [7.25625813e-01 9.22750309e-02 8.38934556e-02 ... 4.44001946e-09
  1.79879734e-07 6.28959333e-06]
 ...
 [9.71989632e-01 2.08351575e-02 5.04804775e-03 ... 1.34528975e-08
  3.20130368e-07 1.78760558e-04]
 [7.96798170e-01 1.25998273e-01 4.99982163e-02 ... 9.67295222e-08
  1.00171303e-06 4.03183309e-04]
 [7.15329826e-01 9.55083519e-02 8.98104608e-02 ... 2.04667239e-09
  8.72931309e-08 2.95446762e-06]]


In [175]:
kerasNLP_model.model_type

'Boston311KerasNLP'

In [176]:
import pandas as pd

#get model_name from json_file name and ml_model_date from json_file name first 8 characters which are YYYYMMDD and change it to YYYY-MM-DD
model_name = json_file.split('/')[-1].split('.')[0]
ml_model_date = model_name[:4] + '-' + model_name[4:6] + '-' + model_name[6:8]

#define an empt pandas dataframe ml_model_df
ml_model_df = pd.DataFrame(columns=['ml_model_name', 'ml_model_type', 'ml_model_date'])

ml_model_df = pd.concat([ml_model_df, pd.DataFrame([{'ml_model_name': model_name, 
                                    'ml_model_type': kerasNLP_model.model_type,
                                    'ml_model_date': ml_model_date}])], ignore_index=True)

print(ml_model_df)

                       ml_model_name      ml_model_type ml_model_date
0  20231012_005720_Boston311KerasNLP  Boston311KerasNLP    2023-10-12


In [177]:
model_cases = data.drop(['geom_4326','survival_time_hours', 'survival_time', 'event'], axis=1).copy()

In [178]:
all_model_cases = model_cases 
all_model_predictions = predictions_df

In [179]:
all_model_predictions['ml_model_name'] = model_name

In [180]:
all_model_predictions['prediction_date'] = today_datestring

In [181]:
# %%
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

In [182]:


# %%
all_model_cases.to_csv(my_datetime+'_311_cases.csv', index=False)


# %%

all_model_predictions.to_csv(my_datetime+'_311_predictions.csv', index=False)

# %%

ml_model_df.to_csv(my_datetime+'_311_ml_models.csv', index=False)

# %%
#create datetime _manifest.txt file with one filename per line
with open(my_datetime+'_manifest.txt', 'w') as f:
    f.write(my_datetime+'_311_cases.csv\n')
    f.write(my_datetime+'_311_predictions.csv\n')
    f.write(my_datetime+'_311_ml_models.csv\n')

# %%
#create an export folder
EXPORT_FOLDER = '~/Documents/BODC-DEI-site/database/seeders'
#copy the csv files to the export folder
!cp {my_datetime}_311_cases.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_predictions.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_ml_models.csv {EXPORT_FOLDER}
!cp {my_datetime}_manifest.txt {EXPORT_FOLDER}



# %% [markdown]
# ** Copy the files to the production server **

# %%
# Define constants for servers
PROD_USER = 'u353344964'
PROD_HOSTNAME = '195.179.236.61'
PORT_NUMBER = 65002
PROD_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel'
STAGE_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel'
PROD_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel/database/seeders'
STAGE_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders'

# %%







In [183]:
import os


def scp_to_server(filename, user=PROD_USER, hostname=PROD_HOSTNAME, port=PORT_NUMBER, export_folder=PROD_EXPORT_FOLDER):
    """Copy a file to the server using scp."""
    command = f"scp -P {port} {filename} {user}@{hostname}:{export_folder}"
    print(f"Executing: {command}")
    os.system(command)

# Use the function to scp files
files_to_copy = [
    f"{my_datetime}_311_cases.csv",
    f"{my_datetime}_311_predictions.csv",
    f"{my_datetime}_311_ml_models.csv",
    f"{my_datetime}_manifest.txt"
]

# Control where to copy
copy_to_prod = True
copy_to_stage = True

for file in files_to_copy:
    if copy_to_prod:
        scp_to_server(file, export_folder=PROD_EXPORT_FOLDER)
    if copy_to_stage:
        scp_to_server(file, export_folder=STAGE_EXPORT_FOLDER)


# %%
PORT_NUMBER

Executing: scp -P 65002 2023-10-14_13-15-04_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders


Executing: scp -P 65002 2023-10-14_13-15-04_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-14_13-15-04_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-14_13-15-04_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-14_13-15-04_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-14_13-15-04_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-14_13-15-04_manifest.txt u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-14_13-15-04_manifest.txt u353344964@195.179.236.61:/

65002

In [184]:
# %%
if copy_to_prod:
    !ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {PROD_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'

if copy_to_stage:
    !ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {STAGE_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'


   INFO  Seeding database.  


Manifest files:
/home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-14_13-15-04_manifest.txt

Total records to process: 46155

Processing /home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-14_13-15-04_311_cases.csv
100 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 27690.
Total records remaining: 46055.
Time for last 100 records: 0.03 seconds.
Estimated time remaining for this file: 9 seconds.
Estimated time for all files: 15 seconds.
200 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 27590.
Total records remaining: 45955.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 2 seconds.
Estimated time for all files: 4 seconds.
300 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 27490.
Total records remaining: 45855.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 1 s